In [1]:
import pandas as pd
import os
import yaml

WORK_DIR = '../'
results_dir = os.path.join(WORK_DIR, 'results', 'estimates')

In [2]:
emb_stats = pd.read_csv(os.path.join(results_dir, 'embeddings_stats.csv'))
emb_stats.rename(columns={'time':'chunk emb time (seconds)','size (bytes)': 'chunk size (bytes)', 'n_samples': 'samples per chunk'}, inplace=True)
emb_stats

,task,model,chunk emb time (seconds),samples per chunk,chunk size (bytes)
0,gene_finding,hyenadna-tiny-1k,1136.719639,4783,16974564567
1,enhancer_annotation,hyenadna-tiny-1k,83.364254,28,1331168444
2,histone_modification,hyenadna-tiny-1k,878.459177,50000,12167916103
3,chromatin_accessibility,hyenadna-tiny-1k,876.718117,50000,12161888991
4,cpg_methylation,hyenadna-tiny-1k,874.093837,50000,12177768906
5,gene_finding,hyenadna-large-1m,1979.431083,4783,33897712652
6,enhancer_annotation,hyenadna-large-1m,148.898188,28,2656704110
7,histone_modification,hyenadna-large-1m,1992.294050,50000,24278876951
8,chromatin_accessibility,hyenadna-large-1m,2031.097446,50000,24272783417
9,cpg_methylation,hyenadna-large-1m,2050.380581,50000,24276876162


In [4]:
downstream_stats = pd.read_csv(os.path.join(results_dir, 'downstream_stats.csv'))
downstream_stats = downstream_stats[downstream_stats['epoch'] == 1].drop(columns=['epoch'])
downstream_stats.rename(columns={'time':'chunk training time (seconds)'}, inplace=True)
downstream_stats

,task,model,chunk training time (seconds)
0,gene_finding,hyenadna-tiny-1k,162.424445
2,enhancer_annotation,hyenadna-tiny-1k,14.042472
4,histone_modification,hyenadna-tiny-1k,120.563790
6,chromatin_accessibility,hyenadna-tiny-1k,121.548147
8,cpg_methylation,hyenadna-tiny-1k,122.220143
10,gene_finding,hyenadna-large-1m,318.331048
12,enhancer_annotation,hyenadna-large-1m,26.340381
14,histone_modification,hyenadna-large-1m,218.308265
16,chromatin_accessibility,hyenadna-large-1m,219.620006
18,cpg_methylation,hyenadna-large-1m,220.473587


In [5]:
df_stats = pd.merge(
    emb_stats,
    downstream_stats,
    on=['model', 'task'],
    how='outer',
    suffixes=('_emb', '_downstream'),
)
df_stats = df_stats.sort_values(by=['task', 'model'])
df_stats

,task,model,chunk emb time (seconds),samples per chunk,chunk size (bytes),chunk training time (seconds)
0,chromatin_accessibility,hyenadna-large-1m,2031.097446,50000,24272783417,219.620006
5,chromatin_accessibility,hyenadna-tiny-1k,876.718117,50000,12161888991,121.548147
10,chromatin_accessibility,nt_transformer_human_ref,3726.499189,50000,21785561139,480.304862
16,chromatin_accessibility,nt_transformer_v2_500m,3995.622332,50000,17348286356,405.769217
21,chromatin_accessibility,resnetlm,4382.098835,50000,24269158218,221.764109
1,cpg_methylation,hyenadna-large-1m,2050.380581,50000,24276876162,220.473587
6,cpg_methylation,hyenadna-tiny-1k,874.093837,50000,12177768906,122.220143
11,cpg_methylation,nt_transformer_human_ref,3621.365921,50000,21774992929,493.871626
17,cpg_methylation,nt_transformer_v2_500m,4021.845386,50000,17350147700,395.007102
22,cpg_methylation,resnetlm,4454.044875,50000,24274541079,219.585632


In [9]:
task_len = {}
for task in df_stats['task'].unique():
    df_task = pd.read_csv(os.path.join(WORK_DIR, 'data',task, f'{task}.bed'), sep='\t', low_memory=False)
    task_len[task] = len(df_task)

df_stats['total samples'] = df_stats['task'].map(task_len)
df_stats['n chunks'] = df_stats['total samples'] / df_stats['samples per chunk']

df_stats['chunk size (GB)'] = df_stats['chunk size (bytes)'] / (1024 ** 3)

df_stats['total emb time (hours)'] = (df_stats['chunk emb time (seconds)'] * df_stats['n chunks']) / 3600
df_stats['total emb size (GB)'] = df_stats['chunk size (bytes)'] * df_stats['n chunks'] / (1024 ** 3)

df_stats['epoch training time (hours)'] = df_stats['chunk training time (seconds)'] * df_stats['n chunks'] / 3600
total_epochs = 100
df_stats['total training time (hours)'] = df_stats['epoch training time (hours)'] * total_epochs
df_stats['total training time (days)'] = df_stats['total training time (hours)'] / 24

df_stats

,task,model,chunk emb time (seconds),samples per chunk,chunk size (bytes),chunk training time (seconds),total samples,n chunks,total emb time (hours),total emb size (GB),epoch training time (hours),total training time (hours),total training time (days),chunk size (GB)
0,chromatin_accessibility,hyenadna-large-1m,2031.097446,50000,24272783417,219.620006,2062129,41.242580,23.268805,932.321150,2.516027,251.602657,10.483444,22.605791
5,chromatin_accessibility,hyenadna-tiny-1k,876.718117,50000,12161888991,121.548147,2062129,41.242580,10.043921,467.139929,1.392489,139.248866,5.802036,11.326642
10,chromatin_accessibility,nt_transformer_human_ref,3726.499189,50000,21785561139,480.304862,2062129,41.242580,42.691789,836.786579,5.502503,550.250325,22.927097,20.289385
16,chromatin_accessibility,nt_transformer_v2_500m,3995.622332,50000,17348286356,405.769217,2062129,41.242580,45.774937,666.350208,4.648603,464.860261,19.369178,16.156851
21,chromatin_accessibility,resnetlm,4382.098835,50000,24269158218,221.764109,2062129,41.242580,50.202517,932.181905,2.540590,254.059001,10.585792,22.602415
1,cpg_methylation,hyenadna-large-1m,2050.380581,50000,24276876162,220.473587,959039,19.180780,10.924416,433.669817,1.174682,117.468205,4.894509,22.609603
6,cpg_methylation,hyenadna-tiny-1k,874.093837,50000,12177768906,122.220143,959039,19.180780,4.657167,217.537495,0.651188,65.118824,2.713284,11.341431
11,cpg_methylation,nt_transformer_human_ref,3621.365921,50000,21774992929,493.871626,959039,19.180780,19.294618,388.977443,2.631345,263.134528,10.963939,20.279542
17,cpg_methylation,nt_transformer_v2_500m,4021.845386,50000,17350147700,395.007102,959039,19.180780,21.428370,309.934249,2.104596,210.459565,8.769149,16.158584
22,cpg_methylation,resnetlm,4454.044875,50000,24274541079,219.585632,959039,19.180780,23.731126,433.628105,1.169951,116.995103,4.874796,22.607428


In [8]:
df_stats.groupby('model').sum(numeric_only=True).sort_values(by='total emb size (GB)', ascending=False)['total emb size (GB)']

model
hyenadna-large-1m           1713.372751
resnetlm                    1713.033909
nt_transformer_human_ref    1536.381586
nt_transformer_v2_500m      1223.715215
hyenadna-tiny-1k             858.756578
nt_transformer_ms             69.616249
Name: total emb size (GB), dtype: float64